# Obligatorio - Deep Learning

El obligatorio consiste en el diseño, entrenamiento y evaluación de Redes Generativas Adversarias (GANs) para la generación de dígitos del estilo de MNIST dataset a partir de un vector de ruido.

Integrantes del equipo:
- Carolina Allende
- Ignacio Aristimuño

## 1. Imports y dependencias

In [1]:
#!source ../obligatorio_env/bin/activate
#!pip install -r requirements.txt

In [2]:
import random
import os
from yaml import safe_load
import matplotlib.pyplot as plt

import torch
from torch import optim
from torch import nn
import torch.backends.cudnn as cudnn

In [3]:
from datasets.mnist_dataset import get_dataloader
from models.dcgan.generator import DCGANGenerator, get_dcgan_generator_defaults
from models.dcgan.discriminator import (
    DCGANDiscriminator,
    get_dcgan_discriminator_defaults,
)
from models.vanilla.generator import VanillaGenerator, get_vanilla_generator_defaults
from models.vanilla.discriminator import (
    VanillaDiscriminator,
    get_vanilla_discriminator_defaults,
)
from training import train

ModuleNotFoundError: No module named 'torchvision'

## 2. Dataset and parameters

In [3]:
# Select GAN model
network = "vanilla"
training_settings = "VanillaGAN"

# Set random seed for reproducibility
seed = 42
random.seed(seed)
torch.manual_seed(seed)

cudnn.benchmark = True

In [4]:
# Settings
with open("config.yaml", "r") as file:
    settings = safe_load(file)


NOISE_DIM = settings["Generator"]["NoiseDim"]

BATCH_SIZE = settings["Dataset"]["BatchSize"]
N_WORKERS = settings["Dataset"]["Workers"]

N_EPOCHS = settings["Training"][training_settings]["N_EPOCHS"]
GENERATOR_LR = settings["Training"][training_settings]["GeneratorLR"]
DISCRIMINATOR_LR = settings["Training"][training_settings]["DiscriminatorLR"]
ADAM_BETA_1 = settings["Training"][training_settings]["AdamBeta1"]

In [5]:
DEVICE = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Current device: {DEVICE.upper()}")

DEVICE = torch.device(DEVICE)

Current device: MPS


In [6]:
dataloader = get_dataloader(BATCH_SIZE, N_WORKERS)

## 3. Networks

### 3.1 Initialization function

In [7]:
def weights_initialization(model):
    classname = model.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(model.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(model.weight.data, 1.0, 0.02)
        nn.init.constant_(model.bias.data, 0)

### 3.2 Generator

In [8]:
# Create Generator
if network == "vanilla":
    generator = VanillaGenerator(**get_vanilla_generator_defaults()).to(DEVICE)
elif network == "DCGAN":
    generator = DCGANGenerator(**get_dcgan_generator_defaults()).to(DEVICE)
    generator.apply(weights_initialization)

print(generator)

DCGANGenerator(
  (sequential): Sequential(
    (0): ConvTranspose2d(64, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SELU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): SELU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): SELU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 1, kernel_size=(1, 1), stride=(1, 1), padding=(2, 2), bias=False)
    (13): Tanh()
  )
)


### 3.3 Discriminator

In [9]:
# Create Discriminator
if network == "vanilla":
    discriminator = VanillaDiscriminator(**get_vanilla_discriminator_defaults()).to(
        DEVICE
    )
elif network == "DCGAN":
    discriminator = DCGANDiscriminator(**get_dcgan_discriminator_defaults()).to(DEVICE)
    discriminator.apply(weights_initialization)

print(discriminator)

DCGANDiscriminator(
  (main): Sequential(
    (0): Conv2d(1, 64, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (9): Sigmoid()
  )
)


## 4. Loss functions and optimizers

In [11]:
# Loss function
criterion = nn.BCELoss()

# Create batch of latent vectors to keep track of training progress of the generator
fixed_noise = torch.randn(64, NOISE_DIM, 1, 1, device=DEVICE)

# Labels' convention
real_label = 1
fake_label = 0

# Optimizers
optimizerG = optim.Adam(
    generator.parameters(), lr=GENERATOR_LR, betas=(ADAM_BETA_1, 0.999)
)
optimizerD = optim.Adam(
    discriminator.parameters(), lr=DISCRIMINATOR_LR, betas=(ADAM_BETA_1, 0.999)
)

## 5. Training

### 5.1 Helper training functions

In [12]:
# Folders for saving results
if network == "vanilla":
    RESULTS_FOLDER = "results/MNIST_vanilla"
elif network == "DCGAN":
    RESULTS_FOLDER = "results/MNIST_DCGAN"

if not os.path.isdir(RESULTS_FOLDER):
    os.makedirs(RESULTS_FOLDER, exist_ok=True)
if not os.path.isdir(f'{RESULTS_FOLDER}/random_results'):
    os.makedirs(f'{RESULTS_FOLDER}/random_results', exist_ok=True)
if not os.path.isdir(f'{RESULTS_FOLDER}/fixed_results'):
    os.makedirs(f'{RESULTS_FOLDER}/fixed_results', exist_ok=True)

### 5.2 Training

In [14]:
train(dataloader, generator, discriminator, DEVICE, optimizerG, optimizerD, criterion, NOISE_DIM, N_EPOCHS, None)

training start!


RuntimeError: Mismatched Tensor types in NNPack convolutionOutput

### 5.3 Plot sample images

In [ ]:
# Testing on fake images
fake_images = torch.randn((25, 100), device=DEVICE).view(-1, 100, 1, 1).detach()

G_result = generator(fake_images)
G_result = G_result.cpu()
G_result = G_result.detach().numpy()
G_result.shape

# setting values to rows and column variables
fig = plt.figure(figsize=(14, 14))
rows = 5
columns = 5

In [ ]:
# Plot results
for k, array in enumerate(G_result):
    ax = plt.subplot(rows, columns, k + 1)
    im = ax.imshow(array.reshape(28, 28))

    plt.tight_layout()
plt.show()